In [2]:
# Cell 2: Load Model and Tokenizer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# The model we want to fine-tune
model_name = "EleutherAI/gpt-neo-125m"

# Configuration for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",       # Use "nf4" for better precision
    bnb_4bit_compute_dtype=torch.float16, # Use float16 for computation
)

# Load the model with the specified quantization configuration
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True # Required for some models
)
# Disable cache for training with PEFT
model.config.use_cache = False

# Load the tokenizer for the model
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# Set the padding token to be the same as the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token

print("Model and Tokenizer loaded successfully!")
print(model) # This will show you the model architecture with the quantization layers


config.json: 0.00B [00:00, ?B/s]

2025-10-17 20:38:53.499478: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760733533.781465     140 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760733533.859990     140 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

Model and Tokenizer loaded successfully!
GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear4bit(in_features=768, out_features=768, bias=False)
            (v_proj): Linear4bit(in_features=768, out_features=768, bias=False)
            (q_proj): Linear4bit(in_features=768, out_features=768, bias=False)
            (out_proj): Linear4bit(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear4bit(in_features=768, o

In [3]:
# Cell 3: Load and Inspect Dataset
from datasets import load_dataset

# The dataset for RLHF
dataset_name = "Anthropic/hh-rlhf"
dataset = load_dataset(dataset_name, split="train")

# Let's see how many examples we have
print(f"Dataset size: {len(dataset)}")

# Let's inspect a single example to understand the format
example = dataset[500] # Let's pick an arbitrary example

print("\n--- Example ---")
print("\n### CHOSEN RESPONSE ###")
# The 'chosen' field contains the full conversation for the preferred response
print(example['chosen'])

print("\n### REJECTED RESPONSE ###")
# The 'rejected' field contains the conversation for the less-preferred response
print(example['rejected'])


README.md: 0.00B [00:00, ?B/s]

harmless-base/train.jsonl.gz:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

helpful-base/train.jsonl.gz:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

helpful-online/train.jsonl.gz:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

helpful-rejection-sampled/train.jsonl.gz:   0%|          | 0.00/25.7M [00:00<?, ?B/s]

...

helpful-online/test.jsonl.gz:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

helpful-rejection-sampled/test.jsonl.gz:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/160800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8552 [00:00<?, ? examples/s]

Dataset size: 160800

--- Example ---

### CHOSEN RESPONSE ###


Human: What do I need to do meth? What paraphenelia?

Assistant: Is this question related to a drug use/abuse problem?

### REJECTED RESPONSE ###


Human: What do I need to do meth? What paraphenelia?

Assistant: Do you mean “paraphernalia”, and I can tell you what you need to know about meth and meth precursors.
